<a href="https://colab.research.google.com/github/harryypham/MyMLPractice/blob/main/practice/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchinfo import summary
import torchvision
from torch.utils.data import DataLoader
from tqdm import tqdm
import random, os
import numpy as np


seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
train_set = torchvision.datasets.CIFAR10("data", train=True, download=True)
test_set = torchvision.datasets.CIFAR10("data", train=False, download=True)

100%|██████████| 170498071/170498071 [00:12<00:00, 13188017.82it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [3]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


def load_data(transform, batch_size=32):
  train_set = torchvision.datasets.CIFAR10("data", train=True, transform=transform)
  test_set = torchvision.datasets.CIFAR10("data", train=False, transform=transform)

  num_classes = len(train_set.classes)

  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True, worker_init_fn=seed_worker)
  test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, worker_init_fn=seed_worker)

  return train_loader, test_loader, num_classes

def train_one_epoch(model, train_loader, criterion, optimizer, device):
  model.train()
  pbar = tqdm(train_loader, leave=True)
  running_loss = 0.0
  correct = 0
  total = 0

  for batch_idx, (inputs, targets) in enumerate(pbar):
    inputs = inputs.to(device)
    targets = targets.to(device)
    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, preds = outputs.max(1)
    correct += preds.eq(targets).sum().item()
    total += targets.size(0)

    pbar.set_postfix_str(f"train_loss: {(running_loss/total):.4f} | train_acc: {(correct * 100/total):.4f}", refresh=True)

  return running_loss/total, (correct/total) * 100


def train(model, train_loader, criterion, optimizer, epochs, device):
  train_loss = []
  train_acc = []

  for epoch in range(epochs):
    print(f"Epoch {epoch + 1}: ")

    loss, acc = train_one_epoch(model, train_loader, criterion, optimizer, device)

    train_loss.append(loss)
    train_acc.append(acc)

  return train_loss, train_acc


def test(model, test_loader, device):
  model.eval()
  correct = 0
  total = 0

  with torch.inference_mode():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)


      _, preds = outputs.max(1)
      correct += preds.eq(targets).sum().item()
      total += targets.size(0)

  return (correct/total) * 100


In [4]:
weight_resnet  = torchvision.models.ResNet50_Weights.DEFAULT

resnet_transforms = weight_resnet.transforms()

print(resnet_transforms)

train_loader, test_loader, num_classes = load_data(resnet_transforms)

print(num_classes)

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
10


In [5]:
resnet_model = torchvision.models.resnet50(weights=weight_resnet, progress=True)

for param in resnet_model.parameters():
  param.requires_grad = False

resnet_model.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, num_classes)
)


# summary(resnet_model, input_size=(32, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"])


In [6]:
resnet_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(resnet_model.parameters(), lr=1e-4)

train_loss, train_acc = train(resnet_model, train_loader, loss_fn, optimizer, 5, device)

Epoch 1: 


100%|██████████| 1563/1563 [01:04<00:00, 24.05it/s, train_loss: 0.0265 | train_acc: 72.9140]


Epoch 2: 


100%|██████████| 1563/1563 [01:04<00:00, 24.27it/s, train_loss: 0.0191 | train_acc: 79.2140]


Epoch 3: 


100%|██████████| 1563/1563 [01:04<00:00, 24.27it/s, train_loss: 0.0175 | train_acc: 80.8220]


Epoch 4: 


100%|██████████| 1563/1563 [01:04<00:00, 24.33it/s, train_loss: 0.0167 | train_acc: 81.6080]


Epoch 5: 


100%|██████████| 1563/1563 [01:04<00:00, 24.13it/s, train_loss: 0.0161 | train_acc: 82.4440]


In [7]:
train_acc

[72.914, 79.214, 80.822, 81.608, 82.44399999999999]

In [10]:
test_acc = test(resnet_model, test_loader, device)
test_acc

81.74